# Project Analysis NoteBook
## Data Input

The datasets used in this project are available from ['The Cancer Genome Atlas' (TCGA)](www.http://cancergenome.nih.gov/) consortium.  Clinical and RNA-seq gene count data sets can be downloaded via a provided data portal, or using an R package called [TCGA2STAT](https://cran.r-project.org/web/packages/TCGA2STAT/index.html).  The R script used to download the data sets for this project are available in the local repository for this project ([GitHub link](https://github.com/CCThompson82/Prostate_metastasis/tree/master/Dataset_setup)).  If run locally, two [feather](https://github.com/wesm/feather) files will be written to the working directory, which will be read and imported with the python scripts below.

In [1]:
%run -i Dataset_cleanup/arrange_DF.py

Clinical data set imported!
Features: 21 
Patients: 499

 The following features do not provide any information: 
 ['Composite.Element.REF' 'ethnicity' 'gender' 'pathologicstage'
 'pathologyMstage' 'tumortissuesite'] 

Variables that are not known at initial diagnosis: 
 ['daystodeath' 'daystolastfollowup' 'daystopsa' 'histologicaltype'
 'numberoflymphnodes' 'pathologyTstage' 'radiationtherapy' 'residualtumor'
 'vitalstatus'] 

Variables that are known at the time of diagnosis:
 ['clinical_index' 'dateofinitialpathologicdiagnosis' 'gleasonscore'
 'pathologyNstage' 'psavalue' 'race' 'yearstobirth']


Gene Counts data set imported!
Features: 20501 
Patients: 497


Transforming gene counts to transcript per million (TPM)

Transformation Successful!

497 Gene count estimate profiles have been transformed from gene counts to transcripts per million reads (TPM)


## Data Exploration
### Clinical Information - including metastasis label
There is missing clinical data in many of the features, including what will become the outcome label ('pathologyNstage' - metastasis state).  The series **'y_all'** is the full list of pathologyNstage, where 'n1' represents metastasis, and 'n0' represents no metastasis observed to date.  Some observations have no metastasis state recorded and are represented by NaN in y_all.  These are removed for the trimmed **'y'** series.  

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
D = (('n0', y_all[y_all =='n0'].shape[0]), 
     ('n1', y_all[y_all =='n1'].shape[0]), 
     ('missing' , y_all[y_all.isnull()].shape[0]))
exp_fig = plt.figure(figsize=(5 , 5))
A = exp_fig.add_subplot(1,1,1)
ind = A.bar(range(0,3), height= [D[0][1], D[1][1], D[2][1]], align='center', color = 'grey')
A.set_xticks(range(0,3))
A.set_ylabel('Frequency', fontsize=16)
A.set_xlabel('Metastasis state', fontsize=16)
A.set_ylim(0,400)
A.set_xticklabels([D[0][0], D[1][0], D[2][0]])  #Must be a better way to do this as tuples (D, in this case) are not ordered?
ep = plt.show
exp_fig.savefig('Figures/Label_count.png')

### Exploratory Visualization

#### Gleason Score versus Metastasis analysis
The Gleason score is the gold-standard diagnostic test for cancer severity, but is not highly correlative with metastatic disease.  

In [ ]:
clinical['gleasonscore'] = pd.to_numeric(clinical['gleasonscore'], errors= 'coerce')
clinical['gleasonscore'].isnull().any()

In [ ]:
GS_hist = plt.figure(figsize=(5,5))
A = GS_hist.add_subplot(1,1,1)
sub = (('n1', ))

subs = [clinical.loc[y_all[y_all == 'n0'].index]['gleasonscore'],
        clinical.loc[y_all[y_all == 'n1'].index]['gleasonscore'],
        clinical.loc[y_all[y_all.isnull()].index]['gleasonscore']]

bins = [5.99, 6.99, 7.99, 8.99, 9.99, 10.99]
A.hist(subs, bins, label =['n0', 'n1', 'unknown'], color = ['blue','red','grey'], stacked = True)
A.set_ylim(0,275)
A.set_ylabel('Count', fontsize=16)
A.set_xlabel('Gleason Score', fontsize =16)
A.legend()
plt.show
GS_hist.savefig('Figures/Gleason_hist.png')

It appears that no metastases are recorded in those specimens graded at a Gleason score of 6.  This represents an opportunity to replace the missing data label with the most likely pathology state, n0, to more efficiently use the small data set.  

In [ ]:
"""Define the indices where gleasonscore == 6 and pathologyNstage is null"""
set(y_all[y_all.isnull()].index).intersection(list(clinical[clinical['gleasonscore'] == 6].index))

In [ ]:
y_all.loc[set(y_all[y_all.isnull()].index).intersection(list(clinical[clinical['gleasonscore'] == 6].index))] = 'n0'

In [ ]:
GS_hist = plt.figure(figsize=(5,5))
A = GS_hist.add_subplot(1,1,1)
sub = (('n1', ))

subs = [clinical.loc[y_all[y_all == 'n0'].index]['gleasonscore'],
        clinical.loc[y_all[y_all == 'n1'].index]['gleasonscore'],
        clinical.loc[y_all[y_all.isnull()].index]['gleasonscore']]

bins = [5.99, 6.99, 7.99, 8.99, 9.99, 10.99]
A.hist(subs, bins, label =['n0', 'n1', 'unknown'], color = ['blue','red','grey'], stacked = True)
A.set_ylim(0,275)
A.set_ylabel('Count', fontsize=16)
A.set_xlabel('Gleason Score', fontsize =16)
A.legend()
plt.show
GS_hist.savefig('Figures/Gleason_hist2.png')

#### Gene Activation (Gene counts) Dataset
The starting dataset, **'X_all'**, includes the transformed transcript per million (TPM) estimates for all RNA-seq profiles.  However some of the observations in this set do not have corresponding y_labels, as the clinical data set contains missing information.

Therefore X_all was trimmed to include only those observations where a finite y label exists, to yield **'X'**.  

In [ ]:
print("Total observations in original dataset:",clinical.shape[0])

not_labeled = y_all[y_all.isnull()] 
y = y_all[y_all.notnull()]

print("\nObservations with metastasis label:",y.shape[0])
print("Unlabeled observations (removed:)",not_labeled.shape[0])


In [ ]:
"""Limit X to only observations where a target label is present."""
X = X_all.loc[set(y.index).intersection(X_all.index)]  #Only observations that also have a known metastasis state are kept.
y = y.loc[set(X.index).intersection(y.index)]
print("X dimensions:",X.shape,"\ny dimensions:",y.shape)

In [ ]:
X_no_y = X_all.loc[list(not_labeled.index)]
print("Dimensions of unlabeled dataset:",X_no_y.shape)

In [ ]:
X.isnull().values.any()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
K_selector = SelectKBest(f_classif, k = 'all') # using k=all here and will filter based on F-stat later.  
K = K_selector.fit(X, y)
K_df = pd.DataFrame({'Gene':X.columns, 'F_score':K.scores_})
K_df.dropna(inplace=True)

In [ ]:
F_fig = plt.figure(figsize=(15,5))
A = F_fig.add_subplot(1,2,1)
A.hist(K_df['F_score'], bins=range(0,45,1), facecolor='green')
#A.set_yscale('log')
A.set_ylabel('Count', fontsize=16)
A.set_xlabel("F-statistic", fontsize=16)
F_fig.suptitle("Distribution of F-statistics for metatastasis state group comparison of gene expression", fontsize = 18)
B = F_fig.add_subplot(1,2,2)
B.hist(K_df['F_score'], bins=range(0,45,1), facecolor='green')
B.set_yscale('log')
B.set_ylabel('Count', fontsize=16)
B.set_xlabel("F-statistic", fontsize=16)
#B.set_title("Distribution of F-statistics for metatastasis state group comparison of gene expression", fontsize = 18)
plt.show
F_fig.savefig("Figures/F_distribution.png")

### Benchmark 
#### Determine the Null Error / Accuracy Rates for Prediction
As the dataset is unbalanced, knowledge of the null rate - i.e. the performance measure given if a model were to predict the positive label indiscriminantly - is an important benchmark upon which to improve.  

In [ ]:
from sklearn.metrics import matthews_corrcoef, fbeta_score, classification_report, log_loss
from sklearn.model_selection import cross_val_score

In [ ]:
"""Scorers needed throughout code:"""
from sklearn.metrics import make_scorer
LL_scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True, needs_threshold=False)
MCC_scorer = make_scorer(matthews_corrcoef, greater_is_better=True, needs_proba=False, needs_threshold=False)
fbeta_scorer = make_scorer(fbeta_score, greater_is_better=True, needs_proba=False, needs_threshold=False, pos_label='n1', beta = 2)

In [ ]:
y_n1 = pd.Series(['n1']*len(y))
print('Model predicts indiscriminantly, "n1"')
print('\nNull F beta: ', fbeta_score(y, y_n1, pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(y, y_n1),"\n")
print(classification_report(y, y_n1, labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(y.replace({'n1':1, 'n0':0}), y_n1.replace({'n1':1, 'n0':0})),"\n")

In [ ]:
gleason = clinical['gleasonscore']
age = pd.to_numeric(clinical['yearstobirth'], errors = 'coerce')
age.fillna(value = np.mean(age), inplace=True)
psa = pd.to_numeric(clinical['psavalue'], errors= 'coerce')
psa.fillna(value = np.mean(psa), inplace =True)
gleason = gleason.loc[y.index]

benchmarkDF = pd.DataFrame({'gleason': gleason,
                            'age':age ,
                            'psa' : psa,
                            'y' : y}, index=X.index)

sm = pd.scatter_matrix(benchmarkDF, 
                       alpha=0.25, 
                       figsize= (10,10), 
                       diagonal = 'kde', 
                       c = benchmarkDF['y'].replace({'n1': 'red','n0':'blue'}), 
                       s = 125)

plt.savefig("Figures/clin_scatter_matrix.png")

In [ ]:
benchmarkDF.drop(['y'], axis=1, inplace = True)


In [ ]:
benchmarkDF = benchmarkDF.reindex(y.index)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import train_test_split

train_k = round(len(y)*0.75)
test_k = len(y) - train_k
seed = 123
folds = 4

In [ ]:
benchmark_train, benchmark_test, bench_y_train, bench_y_test = train_test_split(benchmarkDF, y, 
                                                    test_size= test_k, 
                                                    train_size = train_k,
                                                    random_state = seed,
                                                    stratify = y)

In [ ]:
clf_LR_bm = LogisticRegression(penalty='l2',
                              dual=False,
                              tol=0.0001,
                              C=1,
                              fit_intercept=True,
                              intercept_scaling=1,
                              class_weight='balanced',
                              random_state=123,
                              solver='liblinear',
                              max_iter=100,
                              multi_class='ovr',
                              verbose=0,
                              warm_start=False,
                              n_jobs=1)

clf_LR_bm.fit(benchmark_train, bench_y_train)  #use training set for model learning

bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(benchmarkDF['gleason'], clf_LR_bm.predict_proba(benchmarkDF)[:,1], 
          color = y.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.3,
          s = 25)
A.set_ylabel('Benchmark Logistic Regression Prediction')
A.set_xlabel('Gleason')
plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)

groups = [clf_LR_bm.predict_proba(benchmarkDF.loc[y[y=='n1'].index, :])[:,1], 
          clf_LR_bm.predict_proba(benchmarkDF.loc[y[y=='n0'].index, :])[:,1]]
bins = np.arange(0,1,0.05)
B.hist(groups, label =['n1', 'n0'],  bins = bins, color = ['red','blue'], stacked = True)
B.set_xlabel('Probability of Metastasis')
B.set_ylabel('Bin Count')
A.set_ylim(0,1)
B.legend()
#bench_fig.title("Prediction of Metastasis")
plt.show
bench_fig.savefig('Figures/benchmark.png')

In [ ]:
print(benchmarkDF.columns)
clf_LR_bm.coef_

In [ ]:
"Benchmark Performance on Test dataset"
print(clf_LR_bm)
print('\nNull F beta: ', fbeta_score(bench_y_test, clf_LR_bm.predict(benchmark_test), pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(bench_y_test, clf_LR_bm.predict(benchmark_test)),"\n")
print(classification_report(bench_y_test, clf_LR_bm.predict(benchmark_test), labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(bench_y_test.replace({'n0':0, 'n1':1}), 
         clf_LR_bm.predict_proba(benchmark_test)[:,1]))

## Feature Reduction
### Gini Importance Filter

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_RF = RandomForestClassifier(n_estimators=1000, 
                                criterion='gini', 
                                max_depth=3, 
                                min_samples_split=30, 
                                min_samples_leaf=5, 
                                min_weight_fraction_leaf=0.0, 
                                max_features= 'auto', 
                                max_leaf_nodes=None, 
                                bootstrap=True, 
                                oob_score=False, 
                                n_jobs=1, 
                                random_state=seed, 
                                verbose=0, 
                                warm_start=False, 
                                class_weight='balanced')


In [ ]:
clf_RF.fit(X, y)
Gini_DF = pd.DataFrame({'Gini' :clf_RF.feature_importances_}, index=X.columns).sort_values(by = ['Gini'], axis = 0, ascending = False)
#Gini_DF.reset_index(inplace=True)
print(Gini_DF.head())

In [ ]:
"""Set k number of genes to retain"""
k = 20

In [ ]:
X = X.loc[:,Gini_DF.iloc[0:k].index]

## Scale X to Xs

In [ ]:

from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
Xs = pd.DataFrame(scaler.fit_transform(X), columns = X.columns, index = X.index)


## PCA Transformation

In [ ]:
from sklearn.decomposition import PCA
components = 2

In [ ]:
pca = PCA(n_components = components, whiten=False)
pca.fit(Xs)
Xpc = pd.DataFrame(pca.transform(Xs), columns = range(0,components,1), index = X.index)

In [ ]:
sm = pd.scatter_matrix(Xpc, 
                       alpha=0.5, 
                       figsize= (10,10), 
                       diagonal = 'kde', 
                       c = y.replace({'n1': 'red','n0':'blue'}), 
                       s = 50)
plt.savefig("Figures/PC_components_scatter_matrix.png")

In [ ]:
%run -i 'support files/renders_PCA.py'

In [ ]:
pca_results(Xs, pca)

### Test Train Split

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(Xpc, y, 
                                                    test_size= test_k, 
                                                    train_size = train_k,
                                                    random_state = seed,
                                                    stratify = y)

## LR with GridSearch

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:

clf_LR = LogisticRegressionCV(Cs=10, 
                              fit_intercept=True, 
                              cv=folds, 
                              dual=False, 
                              penalty='l2', 
                              scoring='log_loss', 
                              solver='liblinear', 
                              tol=0.0001, 
                              max_iter=100, 
                              class_weight='balanced', 
                              n_jobs=1, 
                              verbose=0, 
                              refit=True, 
                              intercept_scaling=1.0, 
                              multi_class='ovr', 
                              random_state=seed)
clf_LR.fit(X_train, y_train)

In [ ]:
clf_LR.C_

In [ ]:
"Benchmark Performance on Test dataset"
print(clf_LR)
print('\nNull F beta: ', fbeta_score(y_test, clf_LR.predict(X_test), pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(y_test, clf_LR.predict(X_test)),"\n")
print(classification_report(y_test, clf_LR.predict(X_test), labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(y_test.replace({'n0':0, 'n1':1}), 
         clf_LR.predict_proba(X_test)[:,1]))

In [ ]:
clf_LR.coef_

## LR for Gleason + PC_0

In [ ]:
X_2f = pd.DataFrame({'PC_0': Xpc.loc[:,0], 'gleason': benchmarkDF.loc[:,'gleason']}, index=Xpc.index)
X_2f.shape

In [ ]:
X_3f = Xpc.join(benchmarkDF.loc[:,'gleason'])
X_3f.shape

## 2 Features

In [ ]:
X_train = X_2f.loc[X_train.index,:]
X_test = X_2f.loc[X_test.index,:]

In [ ]:
clf_LR = LogisticRegressionCV(Cs=10, 
                              fit_intercept=True, 
                              cv=folds, 
                              dual=False, 
                              penalty='l2', 
                              scoring='log_loss', 
                              solver='liblinear', 
                              tol=0.0001, 
                              max_iter=100, 
                              class_weight='balanced', 
                              n_jobs=1, 
                              verbose=0, 
                              refit=True, 
                              intercept_scaling=1.0, 
                              multi_class='ovr', 
                              random_state=seed)
clf_LR.fit(X_train, y_train)

In [ ]:
bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(clf_LR_bm.predict_proba(benchmarkDF.loc[X_train.index,:])[:,1],
          clf_LR.predict_proba(X_train)[:,1],
          color = y_train.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.4,
          s = 25)
A.set_ylabel('LR')
A.set_xlabel('Benchmark LR')
#plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)

B.scatter(clf_LR_bm.predict_proba(benchmarkDF.loc[X_test.index,:])[:,1],
          clf_LR.predict_proba(X_test)[:,1],
          color = y_test.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.5,
          s = 25)
A.set_ylim(0,1)
A.set_xlim(0,1)
B.set_ylabel('Model Probability of Metastasis')
B.set_xlabel('Benchmark Probability')
B.set_ylim(0,1)
B.set_xlim(0,1)
#B.legend()
#bench_fig.title("Prediction of Metastasis")
plt.show
bench_fig.savefig('Figures/model_benchmark_comparison_train_test.png')

In [ ]:
bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(benchmarkDF.loc[X_train.index,'gleason'],
          clf_LR.predict_proba(X_train)[:,1],
          color = y_train.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.4,
          s = 25)
A.set_xlabel('Gleason')
A.set_ylabel('Model Probability of Metastasis')
#plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)

B.scatter(benchmarkDF.loc[X_test.index,'gleason'],
          clf_LR.predict_proba(X_test)[:,1],
          color = y_test.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.5,
          s = 25)
A.set_ylim(0,1)
#A.set_xlim(0,1)
B.set_ylabel('Model Probability of Metastasis')
B.set_xlabel('Gleason')
B.set_ylim(0,1)
B.set_xlim(5,11)
#B.legend()
#bench_fig.title("Prediction of Metastasis")
plt.show
bench_fig.savefig('Figures/gleason_model_prob.png')

In [ ]:

print(clf_LR)
print('\nF beta: ', fbeta_score(y_test, clf_LR.predict(X_test), pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(y_test, clf_LR.predict(X_test)),"\n")
print(classification_report(y_test, clf_LR.predict(X_test), labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(y_test.replace({'n0':0, 'n1':1}), 
         clf_LR.predict_proba(X_test)[:,1]))

In [ ]:
clf_LR.coef_

In [ ]:
X_train.columns

In [ ]:
clf_LR.C_

## 3 Features

In [ ]:
X_train = X_3f.loc[X_train.index,:]
X_test = X_3f.loc[X_test.index,:]

In [ ]:
clf_LR = LogisticRegressionCV(Cs=10, 
                              fit_intercept=True, 
                              cv=folds, 
                              dual=False, 
                              penalty='l2', 
                              scoring='log_loss', 
                              solver='liblinear', 
                              tol=0.0001, 
                              max_iter=100, 
                              class_weight='balanced', 
                              n_jobs=1, 
                              verbose=0, 
                              refit=True, 
                              intercept_scaling=1.0, 
                              multi_class='ovr', 
                              random_state=seed)
clf_LR.fit(X_train, y_train)

In [ ]:
bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(clf_LR_bm.predict_proba(benchmarkDF.loc[X_train.index,:])[:,1],
          clf_LR.predict_proba(X_train)[:,1],
          color = y_train.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.4,
          s = 25)
A.set_ylabel('LR')
A.set_xlabel('Benchmark LR')
#plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)

B.scatter(clf_LR_bm.predict_proba(benchmarkDF.loc[X_test.index,:])[:,1],
          clf_LR.predict_proba(X_test)[:,1],
          color = y_test.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.5,
          s = 25)
A.set_ylim(0,1)
A.set_xlim(0,1)
B.set_ylabel('Model Probability of Metastasis')
B.set_xlabel('Benchmark Probability')
B.set_ylim(0,1)
B.set_xlim(0,1)
#B.legend()
#bench_fig.title("Prediction of Metastasis")
plt.show
bench_fig.savefig('Figures/3f_benchmark.png')

In [ ]:
bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(benchmarkDF.loc[X_train.index,'gleason'],
          clf_LR.predict_proba(X_train)[:,1],
          color = y_train.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.4,
          s = 25)
A.set_xlabel('Gleason')
A.set_ylabel('Model Probability of Metastasis')
#plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)

B.scatter(benchmarkDF.loc[X_test.index,'gleason'],
          clf_LR.predict_proba(X_test)[:,1],
          color = y_test.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.5,
          s = 25)
A.set_ylim(0,1)
#A.set_xlim(0,1)
B.set_ylabel('Model Probability of Metastasis')
B.set_xlabel('Gleason')
B.set_ylim(0,1)
B.set_xlim(5,11)
#B.legend()
#bench_fig.title("Prediction of Metastasis")
plt.show
bench_fig.savefig('Figures/3f_gleason.png')

In [ ]:

print(clf_LR)
print('\nF beta: ', fbeta_score(y_test, clf_LR.predict(X_test), pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(y_test, clf_LR.predict(X_test)),"\n")
print(classification_report(y_test, clf_LR.predict(X_test), labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(y_test.replace({'n0':0, 'n1':1}), 
         clf_LR.predict_proba(X_test)[:,1]))

In [ ]:
clf_LR.coef_

In [ ]:
X_train.columns

In [ ]:
clf_LR

In [ ]:
clf_LR.C_

## LDA exploration

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis(solver='svd', 
                                 shrinkage=None, 
                                 priors=None, 
                                 n_components=1, 
                                 store_covariance=False, 
                                 tol=0.0001)

In [ ]:
lda.fit(Xs.loc[X_train.index,:], y_train)

In [ ]:
Xlda = pd.DataFrame({'LDA' : lda.transform(Xs).reshape(1,-1)[0], 'gleason': benchmarkDF.loc[:,'gleason']}, index=Xs.index)

In [ ]:
X_train = Xlda.loc[X_train.index,:]
X_test = Xlda.loc[X_test.index,:]

In [ ]:
clf_LR = LogisticRegressionCV(Cs=10, 
                              fit_intercept=True, 
                              cv=folds, 
                              dual=False, 
                              penalty='l2', 
                              scoring='log_loss', 
                              solver='liblinear', 
                              tol=0.0001, 
                              max_iter=100, 
                              class_weight='balanced', 
                              n_jobs=1, 
                              verbose=0, 
                              refit=True, 
                              intercept_scaling=1.0, 
                              multi_class='ovr', 
                              random_state=seed)
clf_LR.fit(X_train, y_train)

In [ ]:
bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(clf_LR_bm.predict_proba(benchmarkDF.loc[X_train.index,:])[:,1],
          clf_LR.predict_proba(X_train)[:,1],
          color = y_train.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.4,
          s = 25)
A.set_ylabel('LDA Probability')
A.set_xlabel('Benchmark LR')
#plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)

B.scatter(clf_LR_bm.predict_proba(benchmarkDF.loc[X_test.index,:])[:,1],
          clf_LR.predict_proba(X_test)[:,1],
          color = y_test.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.5,
          s = 25)
A.set_ylim(0,1)
A.set_xlim(0,1)
B.set_ylabel('Model Probability of Metastasis')
B.set_xlabel('Benchmark Probability')
B.set_ylim(0,1)
B.set_xlim(0,1)
#B.legend()
#bench_fig.title("Prediction of Metastasis")
plt.show
bench_fig.savefig('Figures/model_benchmark_comparison_train_test.png')

In [ ]:
bench_fig = plt.figure(figsize=(15,5))
A= bench_fig.add_subplot(1,2,1)
B = bench_fig.add_subplot(1,2,2)
A.scatter(benchmarkDF.loc[X_train.index,'gleason'],
          clf_LR.predict_proba(X_train)[:,1],
          color = y_train.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.4,
          s = 25)
A.set_xlabel('Gleason')
A.set_ylabel('Model Probability of Metastasis')
#plt.suptitle('Prognosis of metastasis risk based on features available at presentation', fontsize=20)

B.scatter(benchmarkDF.loc[X_test.index,'gleason'],
          clf_LR.predict_proba(X_test)[:,1],
          color = y_test.replace({'n1':'red', 'n0': 'blue', 'NaN':'grey'}),
          alpha = 0.5,
          s = 25)
A.set_ylim(0,1)
#A.set_xlim(0,1)
B.set_ylabel('Model Probability of Metastasis')
B.set_xlabel('Gleason')
B.set_ylim(0,1)
B.set_xlim(5,11)
#B.legend()
#bench_fig.title("Prediction of Metastasis")
plt.show
bench_fig.savefig('Figures/LDAbyGleason.png')

In [ ]:

print(clf_LR)
print('\nF beta: ', fbeta_score(y_test, clf_LR.predict(X_test), pos_label='n1',beta=2))
print('\nMCC: ',matthews_corrcoef(y_test, clf_LR.predict(X_test)),"\n")
print(classification_report(y_test, clf_LR.predict(X_test), labels = ['n0','n1']))
print('\nLogLoss: ', log_loss(y_test.replace({'n0':0, 'n1':1}), 
         clf_LR.predict_proba(X_test)[:,1]))

In [ ]:
clf_LR.coef_

In [ ]:
X_train.columns

In [ ]:
clf_LR

In [ ]:
clf_LR.C_